In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/turkcehaber/data.csv
/kaggle/input/trstopwordss/tr_stop_word.txt


In [2]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud
import re

/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# **Text Preprocessing**

In [4]:
df=df = pd.read_csv("/kaggle/input/turkcehaber/data.csv")
df.head()

,Haber Gövdesi,Sınıf
0,"Para, şöhret, hepsi var! Peki neden boşanıyorl...",Magazin
1,"Sosyetik güzel Eda Taşpınar, önceki gün Taraby...",Magazin
2,"Suudi Arabistan, İsrail ablukası altındaki Gaz...",Siyaset
3,"MHP Lideri Devlet Bahçeli'den ""Selahattin Demi...",Siyaset
4,"Sıla koronavirüse yakalandı, konserleri ertele...",Magazin


In [5]:
dff=df.copy()

In [6]:
dff.rename(columns={'Haber Gövdesi': 'haber', 'Sınıf': 'sinif'}, inplace=True)

In [7]:
dff.head()

,haber,sinif
0,"Para, şöhret, hepsi var! Peki neden boşanıyorl...",Magazin
1,"Sosyetik güzel Eda Taşpınar, önceki gün Taraby...",Magazin
2,"Suudi Arabistan, İsrail ablukası altındaki Gaz...",Siyaset
3,"MHP Lideri Devlet Bahçeli'den ""Selahattin Demi...",Siyaset
4,"Sıla koronavirüse yakalandı, konserleri ertele...",Magazin


In [8]:
dff.describe(include='all')

,haber,sinif
count,1486009,1486009
unique,1486009,9
top,Siloa Yazıtı'na ilişkin çıkan haberler gerçeği...,Magazin
freq,1,489951


In [9]:
dff.haber.duplicated(keep="first").value_counts()

haber
False    1486009
Name: count, dtype: int64

In [10]:
dff.drop_duplicates(subset="haber",keep="first",inplace=True,ignore_index=True)

In [11]:
dff.describe()

,haber,sinif
count,1486009,1486009
unique,1486009,9
top,Siloa Yazıtı'na ilişkin çıkan haberler gerçeği...,Magazin
freq,1,489951


In [12]:
dff.sinif.unique()

array(['Magazin', 'Siyaset', 'Sağlık', 'Spor', 'Turizm', 'Kültür-Sanat',
       'Finans-Ekonomi', 'Bilim-Teknoloji', 'Çevre'], dtype=object)

In [13]:
dff["sinif"].value_counts()

sinif
Magazin            489951
Siyaset            305452
Spor               230427
Sağlık             122075
Kültür-Sanat       100019
Turizm              71182
Bilim-Teknoloji     68257
Finans-Ekonomi      65414
Çevre               33232
Name: count, dtype: int64

news_dict={'Magazin':0,'Siyaset':1,'Sağlık':2,'Spor':3,'Kültür-Sanat':4,'Turizm':5,'Finans-Ekonomi':6,'Bilim-Teknoloji':7,'Çevre':8}
dff["sinif"]=dff["sinif"].map(news_dict)

# **Analyze Texts**

In [14]:
dff['words'] = [len(x.split()) for x in dff['haber'].tolist()]

In [15]:
dff

,haber,sinif,words
0,"Para, şöhret, hepsi var! Peki neden boşanıyorl...",Magazin,432
1,"Sosyetik güzel Eda Taşpınar, önceki gün Taraby...",Magazin,131
2,"Suudi Arabistan, İsrail ablukası altındaki Gaz...",Siyaset,122
3,"MHP Lideri Devlet Bahçeli'den ""Selahattin Demi...",Siyaset,131
4,"Sıla koronavirüse yakalandı, konserleri ertele...",Magazin,114
...,...,...,...
1486004,Oyuncu Birkan Sokullu Arnavutköy’de görüntül...,Magazin,10
1486005,Türkiye Cumhuriyet Merkez Bankası’nın 200 baz ...,Siyaset,172
1486006,Apple bu yılın sonuna doğru üç adet yeni akıll...,Bilim-Teknoloji,661
1486007,\n\n\n\n\nSüper Lig’in 18. haftasında Kasımpa...,Spor,214


In [16]:
dff['words'].describe()

count   1486009.00
mean        198.64
std         214.68
min           0.00
25%         103.00
50%         137.00
75%         214.00
max        5450.00
Name: words, dtype: float64

In [17]:
dff = dff[~(dff["words"] == 0)]

In [18]:
dff['words'].describe()

count   1486000.00
mean        198.64
std         214.68
min           1.00
25%         103.00
50%         137.00
75%         214.00
max        5450.00
Name: words, dtype: float64

In [19]:
dff.groupby(['sinif'])['words'].describe()

,count,mean,std,min,25%,50%,75%,max
sinif,,,,,,,,
Bilim-Teknoloji,68257.00,361.67,288.13,1.00,197.00,271.00,433.00,5450.00
Finans-Ekonomi,65414.00,398.57,369.44,1.00,131.00,153.00,635.00,4540.00
Kültür-Sanat,100019.00,231.33,255.46,1.00,138.00,153.00,210.00,5051.00
Magazin,489951.00,142.88,156.26,1.00,58.00,103.00,171.00,4587.00
Sağlık,122075.00,321.19,193.89,1.00,163.00,282.00,421.00,3145.00
Siyaset,305450.00,140.97,112.04,1.00,125.00,133.00,141.00,4325.00
Spor,230423.00,229.60,255.64,1.00,114.00,173.00,272.00,4726.00
Turizm,71179.00,153.28,153.31,1.00,106.00,127.00,138.00,4499.00
Çevre,33232.00,156.48,165.50,1.00,117.00,130.00,140.00,3709.00


In [20]:
min_word_size = 100
max_word_size = 1000
print(dff["sinif"][dff['words'] < min_word_size].value_counts())
print(dff["sinif"][dff['words'] > max_word_size].value_counts())

sinif
Magazin            235970
Spor                43307
Siyaset             31971
Turizm              16313
Finans-Ekonomi       8518
Çevre                7680
Kültür-Sanat         6478
Sağlık               3475
Bilim-Teknoloji      1464
Name: count, dtype: int64
sinif
Finans-Ekonomi     4478
Magazin            2608
Bilim-Teknoloji    2285
Kültür-Sanat       2050
Spor               1192
Siyaset             960
Sağlık              893
Turizm              321
Çevre               218
Name: count, dtype: int64


In [21]:
dff= dff[dff['words']>=min_word_size]
dff= dff[dff['words']<=max_word_size]

In [22]:
dff.groupby(['sinif'])['words'].describe()

,count,mean,std,min,25%,50%,75%,max
sinif,,,,,,,,
Bilim-Teknoloji,64508.00,330.56,184.40,100.00,199.00,268.00,413.00,1000.00
Finans-Ekonomi,52418.00,382.08,274.57,100.00,137.00,170.00,614.00,1000.00
Kültür-Sanat,91491.00,212.88,141.30,100.00,140.00,155.00,211.00,1000.00
Magazin,251373.00,210.71,132.56,100.00,127.00,166.00,242.00,1000.00
Sağlık,117707.00,322.11,173.79,100.00,171.00,287.00,422.00,1000.00
Siyaset,272519.00,146.20,69.33,100.00,128.00,134.00,143.00,1000.00
Spor,185924.00,253.23,157.80,100.00,146.00,201.00,300.00,1000.00
Turizm,54545.00,175.80,122.00,100.00,124.00,131.00,145.00,1000.00
Çevre,25334.00,183.36,130.78,100.00,128.00,134.00,146.00,992.00


In [23]:
dff["sinif"].value_counts()

sinif
Siyaset            272519
Magazin            251373
Spor               185924
Sağlık             117707
Kültür-Sanat        91491
Bilim-Teknoloji     64508
Turizm              54545
Finans-Ekonomi      52418
Çevre               25334
Name: count, dtype: int64

In [24]:
df_new= dff[(dff['sinif'] == "Siyaset") & (dff['words'] > 150)]

In [25]:
dff = dff.drop(df_new.index)

In [26]:
df_new = dff[(dff['sinif'] == "Magazin") & (dff['words'] > 200)]

In [27]:
dff = dff.drop(df_new.index)

In [28]:
df_new = dff[(dff['sinif'] == "Spor") & (dff['words'] > 200)]

In [29]:
dff = dff.drop(df_new.index)

In [30]:
df_new = dff[(dff['sinif'] == "Sağlık") & (dff['words'] > 300)]

In [31]:
dff = dff.drop(df_new.index)

In [32]:
df_new=dff[(dff['sinif'] == "Bilim-Teknoloji") & (dff['words'] < 170)]

In [33]:
dff = dff.drop(df_new.index)

In [34]:
df_new = dff[(dff['sinif'] == "Kültür-Sanat") & ((dff['words'] < 120) | (dff['words'] > 220))]

In [35]:
dff = dff.drop(df_new.index)

In [36]:
dff["sinif"].value_counts()

sinif
Siyaset            235590
Magazin            161063
Spor                92600
Kültür-Sanat        66906
Sağlık              62193
Bilim-Teknoloji     55351
Turizm              54545
Finans-Ekonomi      52418
Çevre               25334
Name: count, dtype: int64

In [37]:
dff.groupby(['sinif'])['words'].describe()

,count,mean,std,min,25%,50%,75%,max
sinif,,,,,,,,
Bilim-Teknoloji,55351.00,361.50,181.21,170.00,225.00,296.00,449.00,1000.00
Finans-Ekonomi,52418.00,382.08,274.57,100.00,137.00,170.00,614.00,1000.00
Kültür-Sanat,66906.00,154.74,22.66,120.00,138.00,148.00,166.00,220.00
Magazin,161063.00,140.34,28.09,100.00,116.00,136.00,162.00,200.00
Sağlık,62193.00,191.26,55.06,100.00,143.00,178.00,239.00,300.00
Siyaset,235590.00,132.34,8.92,100.00,127.00,132.00,139.00,150.00
Spor,92600.00,146.95,28.17,100.00,123.00,146.00,170.00,200.00
Turizm,54545.00,175.80,122.00,100.00,124.00,131.00,145.00,1000.00
Çevre,25334.00,183.36,130.78,100.00,128.00,134.00,146.00,992.00


In [38]:
target_classes = ['Siyaset', 'Magazin', 'Spor','Kültür-Sanat','Sağlık','Bilim-Teknoloji','Turizm','Finans-Ekonomi','Çevre']
target_df = dff[dff['sinif'].isin(target_classes)]

# Her bir hedef sınıf için alt örnekleme işlemini gerçekleştirelim
undersampled_df = pd.concat([
    target_df[target_df['sinif'] == 'Siyaset'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Magazin'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Spor'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Kültür-Sanat'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Sağlık'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Kültür-Sanat'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Sağlık'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Bilim-Teknoloji'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Turizm'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Finans-Ekonomi'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Çevre'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    dff[~dff['sinif'].isin(target_classes)]  # Diğer sınıfları olduğu gibi alalım
])

print("\nAlt Örnekleme Sonrası Sınıf Dağılımları:")
print(undersampled_df['sinif'].value_counts())


Alt Örnekleme Sonrası Sınıf Dağılımları:
sinif
Sağlık             50668
Kültür-Sanat       50668
Siyaset            25334
Spor               25334
Magazin            25334
Bilim-Teknoloji    25334
Turizm             25334
Finans-Ekonomi     25334
Çevre              25334
Name: count, dtype: int64


In [39]:
undersampled_df.groupby(['sinif'])['words'].describe()

,count,mean,std,min,25%,50%,75%,max
sinif,,,,,,,,
Bilim-Teknoloji,25334.00,361.29,180.99,170.00,225.00,297.00,449.00,999.00
Finans-Ekonomi,25334.00,381.04,274.24,100.00,137.00,167.00,613.75,1000.00
Kültür-Sanat,50668.00,154.78,22.71,120.00,139.00,148.00,166.00,220.00
Magazin,25334.00,140.58,28.24,100.00,116.00,136.00,162.00,200.00
Sağlık,50668.00,191.00,55.11,100.00,143.00,177.00,239.00,300.00
Siyaset,25334.00,132.39,8.87,100.00,127.00,133.00,139.00,150.00
Spor,25334.00,147.17,28.20,100.00,123.00,146.00,170.00,200.00
Turizm,25334.00,176.66,123.16,100.00,124.00,131.00,146.00,1000.00
Çevre,25334.00,183.36,130.78,100.00,128.00,134.00,146.00,992.00


In [40]:
undersampled_df

,haber,sinif,words
1180862,ü alana yasak gelmesini engellemek için haberi...,Siyaset,130
428660,HDP'li başkan 'örgüt propagandası' suçundan tu...,Siyaset,127
733804,a bahanesiyle organize edildiğini söyledi. Put...,Siyaset,139
189347,Gençlik ve Spor Bakanı Mehmet Muharrem Kasapoğ...,Siyaset,124
794541,CHP'li isimden çarpıcı açıklama! 'Süreç o nokt...,Siyaset,144
...,...,...,...
1374944,eler ile tarımda ciddi bir olumsuz dönüşüm yaş...,Çevre,126
195731,r Türkiye’den Ekoloji Kolektifi üyesi Cem Günd...,Çevre,136
495355,ültesi Öğretim Üyesi Prof. Dr. Sıddık Cindoruk...,Çevre,132
1218658,lık Bakanı’nın kurucusu olduğu Medipol Ünivers...,Çevre,126


In [41]:
def clean_text(text):
    unwanted_pattern = r'[!.\n,\'“”?@#x*-+/%&"\'-:;]'
    regex = re.compile(unwanted_pattern)
    cleaned_text = regex.sub(" ", text)
    cleaned_text = re.sub(r'\d+', '', cleaned_text)  
    cleaned_text = cleaned_text.lower()  
    return cleaned_text

In [42]:
undersampled_df["haber"] = undersampled_df["haber"].apply(clean_text)

In [43]:
undersampled_df

,haber,sinif,words
1180862,ü alana yasak gelmesini engellemek için haberi...,Siyaset,130
428660,hdp li başkan örgüt propagandası suçundan tu...,Siyaset,127
733804,a bahanesiyle organize edildiğini söyledi put...,Siyaset,139
189347,gençlik ve spor bakanı mehmet muharrem kasapoğ...,Siyaset,124
794541,chp li isimden çarpıcı açıklama süreç o nokt...,Siyaset,144
...,...,...,...
1374944,eler ile tarımda ciddi bir olumsuz dönüşüm yaş...,Çevre,126
195731,r türkiye’den ekoloji kolektifi üyesi cem günd...,Çevre,136
495355,ültesi öğretim üyesi prof dr sıddık cindoruk...,Çevre,132
1218658,lık bakanı’nın kurucusu olduğu medipol ünivers...,Çevre,126


In [44]:
word_freq= undersampled_df.haber.str.split(expand=True).stack().value_counts()


In [45]:
word_freq=word_freq.reset_index(name='freq').rename(columns={'index': 'word'})

In [46]:
top_50_frequent_words = word_freq[:50]
top_50_frequent_words

,word,freq
0,ve,1336396
1,bir,925483
2,bu,584461
3,da,406814
4,için,368458
5,de,361762
6,ile,356082
7,olarak,230121
8,daha,202425
9,çok,198729


In [47]:
tr_stop_words = pd.read_csv('/kaggle/input/trstopwordss/tr_stop_word.txt',header=None)
print("First 5 entries:")
for each in tr_stop_words.values[:5]:
  print(each[0])

First 5 entries:
ama
amma
anca
ancak
bu


In [48]:
for each in top_50_frequent_words['word']:
  if each in tr_stop_words.values:
    print (each)

ve
bir
bu
da
için
de
ile
daha
çok
ise
göre
gibi
kadar
sonra
dr
oldu
ancak
ya
ın
a


In [49]:
def remove_stopwords(text):
    words = text.split()
    clean_words = [word for word in words if word.lower() not in tr_stop_words.values]
    return ' '.join(clean_words)

In [50]:
undersampled_df["haber"] =undersampled_df["haber"].apply(remove_stopwords)

In [51]:
undersampled_df = undersampled_df.set_index('words')  


In [52]:
undersampled_df.to_csv('undersampled_data.csv') 